<a href="https://colab.research.google.com/github/JayaKrishanS/Akaike_Technologies_NLP_Assignment/blob/main/voice_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Internship Assignment for NLP(Voice AI)
 By : Jaya Krishna S



**Problem Statement**

The objective of this problem statement is to create a robust model that can accurately transcribe spoken Marathi content.

In [ ]:
! pip3 install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip3 install accelerate

In [ ]:
pip install python-Levenshtein

In [ ]:
pip install jiwer

In [4]:
import zipfile
import os

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import pandas as pd
import jiwer

In [8]:
# Unzipping the Audio files from drive

filename = '/content/drive/MyDrive/Akaike_technologies_NLP_assesment/common_voice_test.zip'

extract_folder = 'common_voice_test'
os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f'{filename} extracted successfully.')

/content/drive/MyDrive/Akaike_technologies_NLP_assesment/common_voice_test.zip extracted successfully.


In [11]:
# Determine the device for computation
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [12]:
# Define the pre-trained model
model_id = "openai/whisper-large-v3"

# Initializing the Speech-to-Text model with the specified model identifier
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    use_safetensors=True
)
model.to(device)

# Processor for the Whisper-Large model
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
# Creating an ASR (Automatic Speech Recognition) pipeline using Hugging Face's transformers library
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,        # Maximum number of tokens to consider for ASR
    chunk_length_s=30,         # Input audio chunk length in seconds
    batch_size=16,             # Batch size for parallel processing
    return_timestamps=True,    # Include timestamps in the output
    torch_dtype=torch_dtype,   # Specify the torch data type for computations
    device=device,             # Computing device (GPU or CPU)
)

In [14]:
# Folder having the audio files
folder = "/content/common_voice_test/common_voice_test"

In [ ]:
# An empty list to store results
result = []
count = 1

# Iterate through each file in the folder
for file_name in os.listdir(folder):
    audio_path = os.path.join(folder, file_name)
    results = pipe(audio_path, generate_kwargs={"language": "marathi"})
    result.append({"audio_file": file_name, "Predicted_text": results["text"]})
    print(count)
    count += 1

# Creating a Pandas DataFrame from the list of results
df = pd.DataFrame(result)

In [20]:
df

,audio_file,Predicted_text
0,common_voice_mr_30633811.wav,तसस पॉल डिरेक यांचे मार्गधर्शना खाली गनीताचा ...
1,common_voice_mr_30620895.wav,गंगा नदीला पवित्र मानल्या में अनेक कवी ने गंग...
2,common_voice_mr_30647875.wav,चंड्रयान एक हे मानव रहित अंतरायान चंड्रया भोत...
3,common_voice_mr_31484199.wav,याज गावाचे नावनंतर वीजय नगर मुनुन रूड जाले
4,common_voice_mr_30623439.wav,आई मेडी त्याच खोलीत बस्तुन रहाईगी
...,...,...
1811,common_voice_mr_31461846.wav,कृष्ण सोडून इतर कोणा राज कोडाला क्याची चिंता ...
1812,common_voice_mr_30773561.wav,साहित्य सम्मेलन मारखोल करांच्या अध्यक्षते खाल...
1813,common_voice_mr_30628543.wav,गांधारी ही मुच्ची गांधारी या देशाची राजकन्यास...
1814,common_voice_mr_31384353.wav,मीनन अथन्ने स्वर्गाद जाऊन वास केला


In [21]:
text_file_path = '/content/trans.txt'

with open(text_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

data = [line.strip().split('\t') for line in lines]

df_trans = pd.DataFrame(data, columns=['audio_file', 'Reference_text'])
df_trans

,audio_file,Reference_text
0,﻿common_voice_mr_32645351.wav,स्वेच्छानिवृतीचा पध्दत मोठ्या प्रमाणात अंमलात ...
1,common_voice_mr_32127661.wav,चार चार विषय आहेत
2,common_voice_mr_32127660.wav,क्रोष्टु सहस्रजित् नल अंतिक व लघु अशी त्याच्या...
3,common_voice_mr_32126825.wav,त्यातील एर्देनि हे रत् नाचे व सुबाशिदि हे सुभा...
4,common_voice_mr_32126698.wav,आदरार्थी शब्दांनी केला जात असे लहानपणी गंगेजवळ...
...,...,...
1811,common_voice_mr_27591999.wav,पंजाब एक कृषि प्रधान राज्य आहे
1812,common_voice_mr_27591990.wav,संत ज्ञानेश्वरांच्या सातव्या जन्मशताब्दीनिमित्...
1813,common_voice_mr_27591988.wav,सत्यवती अंबालिकेला व्यासांना बघितल्यावर डोळे म...
1814,common_voice_mr_27591987.wav,अनेक रचना अभंग गवळणी असे स्फुट लेखन त्यांनी केले


In [22]:
#Merging the reference and predicted dataframes
combined_df = pd.merge(df_trans, df, on='audio_file', how='inner')

In [22]:
combined_df = pd.read_csv("Results.csv")

In [23]:
combined_df

,audio_file,Reference_text,Predicted_text
0,common_voice_mr_32127661.wav,चार चार विषय आहेत,चार चार विशय आहेट
1,common_voice_mr_32127660.wav,क्रोष्टु सहस्रजित् नल अंतिक व लघु अशी त्याच्या...,"प्रोष्टू, सहस्त्रजीत, नलव, अंतिक वलगु अशी त्य..."
2,common_voice_mr_32126825.wav,त्यातील एर्देनि हे रत् नाचे व सुबाशिदि हे सुभा...,क्यातिल एदेरनी हे रतनाचे व सुभाशिदी हे सुभाशि...
3,common_voice_mr_32126698.wav,आदरार्थी शब्दांनी केला जात असे लहानपणी गंगेजवळ...,आदरार्थी शब्दानी केला जात असे लहान पणी गंगे ज...
4,common_voice_mr_32121632.wav,खमंग वास येतोय,परमंग वासी यूथे
...,...,...,...
1810,common_voice_mr_27591999.wav,पंजाब एक कृषि प्रधान राज्य आहे,पंजाब एक रुषी प्रदान राज्या है।
1811,common_voice_mr_27591990.wav,संत ज्ञानेश्वरांच्या सातव्या जन्मशताब्दीनिमित्...,संत न्यानेश्वरांच्या सात्व्या जन्म स्यताब्दि ...
1812,common_voice_mr_27591988.wav,सत्यवती अंबालिकेला व्यासांना बघितल्यावर डोळे म...,सत्यवती अम्बालिकेला व्यासाना बगितल्यावर डोए म...
1813,common_voice_mr_27591987.wav,अनेक रचना अभंग गवळणी असे स्फुट लेखन त्यांनी केले,"अनेक रचना, अभंग, गवलनी असे स्पूट लेकन त्याननी..."


In [24]:
pred_table = combined_df.copy()

In [25]:
combined_df.head(10)

,audio_file,Reference_text,Predicted_text
0,common_voice_mr_32127661.wav,चार चार विषय आहेत,चार चार विशय आहेट
1,common_voice_mr_32127660.wav,क्रोष्टु सहस्रजित् नल अंतिक व लघु अशी त्याच्या...,"प्रोष्टू, सहस्त्रजीत, नलव, अंतिक वलगु अशी त्य..."
2,common_voice_mr_32126825.wav,त्यातील एर्देनि हे रत् नाचे व सुबाशिदि हे सुभा...,क्यातिल एदेरनी हे रतनाचे व सुभाशिदी हे सुभाशि...
3,common_voice_mr_32126698.wav,आदरार्थी शब्दांनी केला जात असे लहानपणी गंगेजवळ...,आदरार्थी शब्दानी केला जात असे लहान पणी गंगे ज...
4,common_voice_mr_32121632.wav,खमंग वास येतोय,परमंग वासी यूथे
5,common_voice_mr_32121623.wav,चारुता सागर दिनकर दत्तात्रेय भोसले,चारुता सागतिनकर दत्तात्रे बोसली
6,common_voice_mr_32023928.wav,ही फेलोशिप मिळवणारे ते पहिले भारतीय होत,यी फेलोशिप मिलोनारे ते पहिले भारतिया होत।
7,common_voice_mr_32022227.wav,तिचे सूत्रधार शंकरराव देव होते,तीचे सुत्रदार शंकरराव देव होते।
8,common_voice_mr_31931304.wav,चला आज फिरायला जाऊयात,चला आज फिराईला जाऊयात।
9,common_voice_mr_31928736.wav,मध्य भागातील पिवळ्या रंगावर वंदे मातरम् असे लि...,मध्य भागातिल प्यूल्या रंगावर वन्दे मात्रम असे...


In [26]:
import Levenshtein

In [27]:
def calculate_cer(reference, predicted):
    return Levenshtein.distance(reference, predicted) / max(len(reference), len(predicted), 1)

# Apply the function to each row and create a new 'CER' column
combined_df['CER'] = combined_df.apply(lambda row: calculate_cer(row['Reference_text'], row['Predicted_text']), axis=1)

In [28]:
combined_df['Average_CER'] = combined_df['CER'].mean()

In [29]:
combined_df

,audio_file,Reference_text,Predicted_text,CER,Average_CER
0,common_voice_mr_32127661.wav,चार चार विषय आहेत,चार चार विशय आहेट,0.166667,0.230683
1,common_voice_mr_32127660.wav,क्रोष्टु सहस्रजित् नल अंतिक व लघु अशी त्याच्या...,"प्रोष्टू, सहस्त्रजीत, नलव, अंतिक वलगु अशी त्य...",0.232877,0.230683
2,common_voice_mr_32126825.wav,त्यातील एर्देनि हे रत् नाचे व सुबाशिदि हे सुभा...,क्यातिल एदेरनी हे रतनाचे व सुभाशिदी हे सुभाशि...,0.205479,0.230683
3,common_voice_mr_32126698.wav,आदरार्थी शब्दांनी केला जात असे लहानपणी गंगेजवळ...,आदरार्थी शब्दानी केला जात असे लहान पणी गंगे ज...,0.135417,0.230683
4,common_voice_mr_32121632.wav,खमंग वास येतोय,परमंग वासी यूथे,0.500000,0.230683
...,...,...,...,...,...
1810,common_voice_mr_27591999.wav,पंजाब एक कृषि प्रधान राज्य आहे,पंजाब एक रुषी प्रदान राज्या है।,0.281250,0.230683
1811,common_voice_mr_27591990.wav,संत ज्ञानेश्वरांच्या सातव्या जन्मशताब्दीनिमित्...,संत न्यानेश्वरांच्या सात्व्या जन्म स्यताब्दि ...,0.178571,0.230683
1812,common_voice_mr_27591988.wav,सत्यवती अंबालिकेला व्यासांना बघितल्यावर डोळे म...,सत्यवती अम्बालिकेला व्यासाना बगितल्यावर डोए म...,0.134328,0.230683
1813,common_voice_mr_27591987.wav,अनेक रचना अभंग गवळणी असे स्फुट लेखन त्यांनी केले,"अनेक रचना, अभंग, गवलनी असे स्पूट लेकन त्याननी...",0.211538,0.230683


In [30]:
# Converting the output to csv file
csv_filename = 'Output.csv'
combined_df.to_csv(csv_filename, index=False)